In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import requests
import re

In [2]:
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(chrome_options)

In [3]:
url = "https://careers.factorialhr.com"
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
offers = soup.find_all(class_='outline-none text-center inline-block font-bold leading-xs rounded-sm text-xs px-6 py-2 buttonSecondary')

In [16]:
links = [offer.get('href') for offer in offers if 'job_posting' in offer.get('href')]

In [50]:
offers_to_db = {"offers":[]}

for l in links:
    driver.get(l)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    conditions = {i.text.strip() for i in soup.find_all(class_='inline-block align-middle mr-2 ml-2')}
    offer = [i.text for i in soup.find_all(class_=re.compile(r'TextEditorTheme__paragraph|TextEditorTheme__listItem'))]
    offers_to_db["offers"].append({"offer": offer,
                                   "conditions":list(conditions)})
    time.sleep(2)

In [51]:
import json

with open("factorial-offers.json", 'w') as fout:
    json.dump(offers_to_db, fout, indent=2)

In [52]:
import json
import uuid
from azure.cosmos import exceptions, CosmosClient, PartitionKey
import os 
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("UDEMY_TRAININGS_ENDPOINT")
key = os.getenv("UDEMY_TRAININGS_PRIMARY_KEY")
client = CosmosClient(endpoint, key)

database_name = os.getenv("UDEMY_TRAININGS_DATABASE")
container_name = os.getenv("FACTORIAL_CAREERS_CONTAINER")
database = client.create_database_if_not_exists(id=database_name)
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path="/id"),
    offer_throughput=400
)

with open('factorial-offers.json') as f:
    data = json.load(f)

for offer in data['offers']:
    offer['id'] = str(uuid.uuid4())
    container.create_item(body=offer)